# Variable Calculation

This Jupyter Notebook enables the calculation of additional variables from the given dataset which can be further used in the model selection & model training.

The input needed for this script is a dataset containing weather variables for one or more locations. The first column of the dataframe should contain the day of observation in the format of YYYY-MM-DD. Further, the dataframe should contain a column called 'city' that contains the name of the weather station. The following naming convention is applied for weather variables here:

- ws: windspeed
- t: temperature
- wd: wind direction in degrees
- h: humidity relative to temperature
- p: precipitation
- sun: solar irradiance
- press: air pressure
- dew: dew point

Not all of these variables have to be supplied. 


## Load Libraries & Data

In [10]:
# import libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import math
from sklearn.decomposition import PCA, KernelPCA

In [15]:
# input path to raw csv file with air pollution and weather data
data_path = './che/df_che.csv'

In [16]:
# read in dataset
df = pd.read_csv(data_path, encoding='latin1', index_col = 0)

In [4]:
# get an overview of the variables 
print('The dataset contains the following variables:', df.columns.values)

The dataset contains the following variables: ['pm10' 'sun' 't' 'o3' 'no2' 'nox' 'no' 'ws' 'wd' 'press' 'h' 'pm25' 'co'
 'p' 'so2' 'city']


In [5]:
# set index to datetime format value of time column & drop time column
df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')

In [6]:
# get vector with names of all the cities
cities = np.unique(df['city'].values)
print('The dataset covers the following cities:', cities)

The dataset covers the following cities: ['Opfikon_Balsberg' 'StGallen_Blumenbergplatz' 'StGallen_Stuelegg'
 'Zuerich_Schimmelstrasse' 'Zuerich_Stampfenbachstrasse']


## Calculate Additional Variables

### Dew Point
The dew point indicates the amount of moisture in the air. The higher the dew point, the higher the moisture content of the air at a given temperature. If not included in the dataset, the dew point can approximated using the following simple formula:

$$
t_{dp} = t - \frac{100- h}{5}
$$

where $t_{dp}$ is the dew point, $t$ is the temperature and $h$ is relative humidity.


In [7]:
if not('dew' in df.columns):
    df['dew'] = df['t'] -((100-df['h'])/5)

## Principal Component Analysis
Prinicipal Component Analysis (PCA) is a technique to reduce dimensionality. As humidity, temperature, dew point and precipation are often highly correlated (due to the definition of these variables or due to environmental relationships), PCA offers a technique to create a variable that captures most of the variablity present in these variables. This can help reduce collinearity in the final model (which in turn can lead to incorrect attribution of effects to variables). The first component calculated through PCA is added to the dataset with the name 'pca'.

For more information on collinearity: https://doi.org/10.1111/j.1600-0587.2012.07348.x

In [8]:
# perform principal component analysis for humidity, dew point, temperature and precipitation (if all of these are available)

# initalize PCA model & fit to dataset
n = 0
pca_vars = ['h', 'dew', 't', 'p']
for var in pca_vars:
    if var in df.columns:
        n = n + 1
    else:
        pca_vars.remove(var)

pca=PCA(n_components = n) 
_pca = pca.fit(df[pca_vars]) 

# get one transformed variable
transformed = pca.fit_transform(df[pca_vars])[:,0]
df['pca'] = transformed

Calculation of various long-term lag variables for the first principal component variable.

In [9]:
df['lagpca_year'] =df.pca.rolling(window=336, min_periods =1).mean()
df['lagpca_halfyear'] =df.pca.rolling(window=168, min_periods =1).mean()
df['lagpca_12weeks'] = df.pca.rolling(window=84, min_periods =1).mean()
df['lagpca_8weeks'] = df.pca.rolling(window=56, min_periods =1).mean()
df['lagpca_4weeks'] = df.pca.rolling(window=28, min_periods =1).mean()
df['lagpca_2weeks'] = df.pca.rolling(window=14, min_periods =1).mean()
df['lagpca_1week'] = df.pca.rolling(window=7, min_periods =1).mean()

## Long-Term Windspeed Lags
As wind speed is known to have a big influence on air pollution in general and often in many cities the current air pollution levels are also dependent on the wind speeds the weeks before, laggeed variables with the mean and maximum windspeed through the last 1 week, 2 weeks and 4weeks are calculated. 

In [10]:
df['lagws_4weeks'] = df.ws.rolling(window=28, min_periods =1).mean()
df['lagws_2weeks'] = df.ws.rolling(window=14, min_periods =1).mean()
df['lagws_1week'] = df.ws.rolling(window=7, min_periods=1).mean()

df['lagws_4weeks_max'] = df.ws.rolling(window=28, min_periods =1).max()
df['lagws_2weeks_max'] = df.ws.rolling(window=14, min_periods =1).max()
df['lagws_1week_max'] = df.ws.rolling(window=7, min_periods=1).max()

## Year, Month & Weekday

Additional categorical variables are added for the year, month and weekday of an observation.

In [11]:
# calculate additional time variables
df['year'] = df.index.year
df['weekday'] = df.index.weekday
df['month'] = df.index.month

## Winddirection Transformation
The wind direction variable is ransformed from a linear scale of wind direction with polar coordinates (0-360°) to Cartesian coordinates, where angles increase clockwise. This allows for a more accurate feature representation of the data for wind direction around the north axis. We call the resulting variables representing x- and y- coordinates of the wind direction 'wx' and 'wy'.

In [12]:
df['wx'] = np.sin(df['wd']/360 * 2*math.pi)
df['wy'] = np.cos(df['wd']/360 * 2*math.pi) 

## Julian Day
Julian Day refers to an ongoing counter variable. The first day of observations in the dataset is assigned the value '0'.

In [13]:
df['julian'] = pd.Series({x: (x - df.index[0]).total_seconds()/3600 for x in df.index})

## Lagged Variables
Often not only the current weather situation influences air pollution but also the weather of the previous days. Therefore, 1-day, 2-days and 3-days lags are caluclated for all weather variables.

In [14]:
# shfit values one period
df['h_lag1'] = df['h'].shift()
df['t_lag1'] = df['t'].shift()
df['wx_lag1'] = df['wx'].shift()
df['wy_lag1'] = df['wy'].shift()
df['ws_lag1'] = df['ws'].shift()
df['dew_lag1'] = df['dew'].shift()

In [15]:
# shift values two periods
df['h_lag2'] = df['h'].shift(periods =2)
df['t_lag2'] = df['t'].shift(periods =2)
df['wx_lag2'] = df['wx'].shift(periods =2)
df['wy_lag2'] = df['wy'].shift(periods =2)
df['ws_lag2'] = df['ws'].shift(periods =2)
df['dew_lag2'] = df['dew'].shift(periods =2)

In [16]:
# shift values three periods
df['h_lag3'] = df['h'].shift(periods =3)
df['t_lag3'] = df['t'].shift(periods =3)
df['wx_lag3'] = df['wx'].shift(periods =3)
df['wy_lag3'] = df['wy'].shift(periods =3)
df['ws_lag3'] = df['ws'].shift(periods =3)
df['dew_lag3'] = df['dew'].shift(periods = 3)

This dataframe with additional variables can be used to perform the model selection algorithm. Details are given in the script "Model Selection".

In [19]:
df.to_csv('.che/df_che2.csv')